In [68]:
import sentencepiece as spm
import pandas as pd
# import urllib.request
import csv
import tqdm
import pickle
from math import log

In [43]:
with open('../Data/brunch/sentence_pickles/dict_sentences_210621-removed-1.pickle', 'rb') as f:
    tmp = pickle.load(f)

In [44]:
sentences = []
for topic in tmp.keys():
    for user_id in tmp[topic].keys():
        for sentence in tmp[topic][user_id]:
            sentences.append(sentence)

In [45]:
len(sentences)

151733

In [46]:
all_sentences = '\n'.join(sentences)

In [47]:
stop_list = [
    '\ude0a',
    '\udc9e',
    '\udc8c',
    '\udf1e'
]

In [48]:
for stop_word in stop_list:
    all_sentences = all_sentences.replace(stop_word, '')

In [49]:
with open('../Data/brunch/sentencepiece_inputs/brunch.txt', 'w', encoding='utf8') as f:
    f.write(all_sentences)

* hyperparameter 설정

In [50]:
templates= '--input={} \
--pad_id={} \
--bos_id={} \
--eos_id={} \
--unk_id={} \
--model_prefix={} \
--vocab_size={} \
--character_coverage={} \
--model_type={}'

train_input_file = "../Data/brunch/sentencepiece_inputs/brunch.txt"
pad_id=0  #<pad> token을 0으로 설정
vocab_size = 50000 # vocab 사이즈
prefix = 'brunch' # 저장될 tokenizer 모델에 붙는 이름
bos_id=1 #<start> token을 1으로 설정
eos_id=2 #<end> token을 2으로 설정
unk_id=3 #<unknown> token을 3으로 설정
character_coverage = 0.9995 # to reduce character set 
model_type ='bpe' # Choose from unigram (default), bpe, char, or word

In [51]:
cmd = templates.format(train_input_file,
                pad_id,
                bos_id,
                eos_id,
                unk_id,
                prefix,
                vocab_size,
                character_coverage,
                model_type)

In [52]:
spm.SentencePieceTrainer.Train(cmd)

* 위의 코드에서 'model_prefix'.model, 'model_prefix'.vocab 두 파일 생성

* vacab 파일을 불러와, 학습된 서브워드들 확인

In [53]:
vocab_list = pd.read_csv('brunch.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)

In [54]:
vocab_list.sample(20)

,0,1
46062,▁유서,-46058
47021,▁달성도,-47017
33779,▁보스가,-33775
49515,둬,-49511
45468,천재,-45464
38654,내기도,-38650
45469,청은,-45465
41595,▁낙담,-41591
8491,▁직장인들은,-8487
28437,부에서는,-28433


* model load

In [55]:
sp = spm.SentencePieceProcessor()
vocab_file = "brunch.model"
sp.load(vocab_file)

True

In [56]:
print(sentence)
print(sp.encode_as_pieces(sentence))
print(sp.encode_as_ids(sentence))

그러니 너무 고민하지 말고 자신의 특성에 맞는 회사에 들어가 사회 생활을 시작해 보는 것이 어떨까.
['▁그러니', '▁너무', '▁고민하지', '▁말고', '▁자신의', '▁특성에', '▁맞는', '▁회사에', '▁들어가', '▁사회', '▁생활을', '▁시작해', '▁보는', '▁것이', '▁어떨까', '.']
[2806, 337, 22428, 1442, 492, 18122, 1240, 827, 1324, 423, 2189, 6853, 1126, 107, 4333, 48380]


In [60]:
sentences_to_ids = []
for sentence in tqdm.tqdm(sentences):
    a = sp.encode_as_ids(sentence)
    sentences_to_ids.append(a)

100%|██████████| 151733/151733 [00:05<00:00, 25512.66it/s]


In [69]:
N = len(sentences) # 총 문장의 수

def tf(t, d):
    return d.count(t)

def idf(t):
    df = 0
    for sentence_to_id in sentences_to_ids:
        df += t in sentence_to_id
    return log(N/(df + 1))

def tfidf(t, d):
    return tf(t,d)* idf(t)

In [70]:
vocab = list(range(50000))

In [71]:
result = []
for j in tqdm.tqdm(range(len(vocab))):
    t = vocab[j]
    result.append(idf(t))

100%|██████████| 50000/50000 [44:52<00:00, 18.57it/s]


In [73]:
idf_ = pd.DataFrame(result, index = vocab, columns = ["IDF"])

In [102]:
idf_.to_csv('./sp_tokenizer_idf.csv', index=None, )

In [75]:
sorted_idf = idf_.sort_values(by=["IDF"])

In [79]:
for idx, row in sorted_idf.iterrows():
    a = sp.decode([idx])
    print(row['IDF'], a)

0.08403629089851601 .
1.1929159430998288 ,
2.14067927015318 수
2.7002249932786246 있는
2.803027614810023 그
2.8547844537190232 ?
2.9383153981155674 있다
2.988332201044296 의
3.03192198940315 이
3.061323635755979 한
3.06583767925119 에
3.077928005033233 '
3.094812741256104 더
3.103289845513512 '
3.1361139171738803 하는
3.1777704779578175 )
3.1871438089570114 이
3.2154743155832373 을
3.2444618524564897 도
3.286404318960613 한다
3.3146507445995774 것
3.3461479610706983 은
3.3480209715452207 내가
3.3669465931970888 할
3.4187025571965055 (
3.440055681667075 것이다
3.4452076765765027 내
3.5021716513724788 것이
3.508314715886193 나는
3.5196017672170203 잘
3.5251815160980886 는
3.5325947815435 과
3.534626155676185 하고
3.5427931702179634 에서
3.563507374605526 가
3.572853237023764 ‘
3.5787390692010255 그리고
3.6007022342097783 때
3.620939423691402 대한
3.632832527205354 만
3.6614022873045826 를
3.6902842219812113 하지만
3.6921338723962474 ’
3.693192353574723 다른
3.693722014604055 하고
3.7337165370042777 것은
3.734819985392103 으로
3.7568663645622085

5.733433548492659 대로
5.7354722851825075 분
5.7354722851825075 사는
5.7354722851825075 히
5.7354722851825075 개인
5.737515186812308 아니면
5.737515186812308 않으면
5.737515186812308 받을
5.739562270434033 요즘
5.739562270434033 쉽지
5.741613553204591 불
5.743669052386687 마치
5.745728785349697 C
5.745728785349697 총
5.747792769570548 s
5.749861022634607 빨리
5.749861022634607 려
5.749861022634607 때도
5.75193356223658 충분히
5.754010406181419 미
5.7581770788762645 식
5.7581770788762645 관리
5.760266943795724 마
5.760266943795724 아닐까
5.762361185398839 당연히
5.76445982205576 문화
5.766562872252539 만들
5.766562872252539 운영
5.768670354592103 성과
5.770782287795247 아는
5.770782287795247 상
5.770782287795247 공
5.770782287795247 든
5.772898690701624 오래
5.772898690701624 대부분의
5.772898690701624 하는데
5.775019582270763 그의
5.775019582270763 다만
5.775019582270763 의미
5.775019582270763 들에게
5.777144981583076 마음
5.777144981583076 속에서
5.777144981583076 있으면
5.777144981583076 인사
5.779274907840901 하
5.781409380369533 하던
5.783548418618283 경우도
5.785692042

6.216144870777811 곳에서
6.216144870777811 느끼는
6.216144870777811 있다
6.219450658912311 er
6.219450658912311 있는가
6.219450658912311 확인
6.219450658912311 대체
6.219450658912311 어야
6.219450658912311 후에
6.219450658912311 든다
6.222767411538304 보고
6.222767411538304 도움을
6.222767411538304 기억
6.222767411538304 일반
6.226095201630979 벌
6.226095201630979 국
6.2294341028964935 사람들에게
6.2294341028964935 갈
6.2294341028964935 앞에서
6.2294341028964935 직업
6.2294341028964935 였던
6.2294341028964935 현
6.232784189781776 구성원
6.232784189781776 리
6.232784189781776 것인가
6.232784189781776 무엇보다
6.232784189781776 모여
6.232784189781776 상황에서
6.232784189781776 어렵다
6.2361455374844805 모르겠다
6.2361455374844805 냐
6.2361455374844805 문
6.2361455374844805 기준
6.2361455374844805 먹
6.239518221963119 경
6.239518221963119 반
6.239518221963119 순
6.239518221963119 게다가
6.239518221963119 P
6.24290231994736 낮은
6.24290231994736 하니
6.24290231994736 많았다
6.24290231994736 년을
6.24290231994736 려고
6.24290231994736 레
6.24290231994736 살고
6.24290231994736 없지만
6.2

6.5227059048270615 판단
6.5227059048270615 관점에서
6.5227059048270615 주말
6.527200294414901 개의
6.527200294414901 팀을
6.527200294414901 연결
6.527200294414901 식으로
6.527200294414901 했다는
6.527200294414901 력이
6.527200294414901 적극적으로
6.527200294414901 것
6.527200294414901 맞지
6.527200294414901 맛있는
6.527200294414901 정확히
6.531714974769427 8
6.531714974769427 옆에
6.531714974769427 L
6.531714974769427 내는
6.531714974769427 i
6.531714974769427 장은
6.531714974769427 에
6.531714974769427 간다
6.531714974769427 뿐만
6.531714974769427 노동
6.531714974769427 았다
6.531714974769427 G
6.531714974769427 되어야
6.531714974769427 릴
6.536250129934818 깊은
6.536250129934818 카페
6.536250129934818 공간
6.536250129934818 님이
6.536250129934818 일상
6.536250129934818 커피
6.536250129934818 군
6.536250129934818 이와
6.536250129934818 가고
6.536250129934818 나면
6.540805946470679 혁신
6.540805946470679 왜
6.540805946470679 식사를
6.540805946470679 행복
6.540805946470679 많습니다
6.540805946470679 부분은
6.540805946470679 직장에서
6.540805946470679 )'
6.540805946470679 결코
6.5

6.748094125995095 개발자
6.748094125995095 입고
6.748094125995095 이야기는
6.748094125995095 근처
6.748094125995095 자연
6.748094125995095 동네
6.748094125995095 들면
6.748094125995095 점이
6.753727943713351 태
6.753727943713351 H
6.753727943713351 보자
6.753727943713351 개의
6.753727943713351 허
6.753727943713351 물을
6.753727943713351 그녀의
6.753727943713351 놓고
6.753727943713351 아니라는
6.753727943713351 그만
6.753727943713351 기존의
6.753727943713351 아닐
6.753727943713351 10
6.753727943713351 소리를
6.753727943713351 없다면
6.759393681249028 목
6.759393681249028 있을지
6.759393681249028 변
6.759393681249028 학생
6.759393681249028 보냈다
6.759393681249028 생각해보면
6.759393681249028 메
6.759393681249028 팀원
6.759393681249028 단계
6.759393681249028 포함
6.759393681249028 했어요
6.759393681249028 지켜
6.759393681249028 찾기
6.759393681249028 스에
6.759393681249028 수록
6.759393681249028 백
6.765091702363666 사와
6.765091702363666 전부터
6.765091702363666 비용을
6.765091702363666 그나마
6.765091702363666 가운데
6.765091702363666 라서
6.765091702363666 본다
6.765091702363666 폭
6.

...

9.850436134607344 증도
9.850436134607344 BM
9.850436134607344 주실
9.850436134607344 베네
9.850436134607344 중얼
9.850436134607344 준은
9.850436134607344 혜리
9.850436134607344 호괘
9.850436134607344 호로
9.850436134607344 볼륨
9.850436134607344 줬어
9.850436134607344 헤어
9.850436134607344 전업
9.850436134607344 이탈
9.850436134607344 ces
9.850436134607344 자코
9.850436134607344 읽어
9.850436134607344 반사
9.850436134607344 인류
9.850436134607344 enn
9.850436134607344 작에
9.850436134607344 인걸
9.850436134607344 삼국
9.850436134607344 아기자기
9.850436134607344 irt
9.850436134607344 이영
9.850436134607344 이듯
9.850436134607344 led
9.850436134607344 인터널
9.850436134607344 80%
9.850436134607344 ava
9.850436134607344 작전
9.850436134607344 김영철
9.850436134607344 ?!)
9.850436134607344 Big
9.850436134607344 전드
9.850436134607344 CES
9.850436134607344 DIS
9.850436134607344 따지
9.850436134607344 eedback
9.850436134607344 제너럴
9.850436134607344 장천
9.850436134607344 rateg
9.850436134607344 Pay
9.850436134607344 TER
9.850436134607344 You
9.850436

9.983967527231867 사료를
9.983967527231867 월엔
9.983967527231867 것이든
9.983967527231867 세균
9.983967527231867 원리
9.983967527231867 사들인
9.983967527231867 운틴
9.983967527231867 뽑히는
9.983967527231867 칼국
9.983967527231867 청장
9.983967527231867 감자즙
9.983967527231867 감가상
9.983967527231867 추방
9.983967527231867 Sur
9.983967527231867 칼로
9.983967527231867 칭찬
9.983967527231867 치려
9.983967527231867 출연
9.983967527231867 지랄
9.983967527231867 가습기
9.983967527231867 가정용
9.983967527231867 평준
9.983967527231867 사이판
9.983967527231867 빨고
9.983967527231867 갈라타
9.983967527231867 진정
9.983967527231867 가야금
9.983967527231867 ough
9.983967527231867 성능에
9.983967527231867 opus
9.983967527231867 스탠다
9.983967527231867 타이핑
9.983967527231867 타임이
9.983967527231867 gue
9.983967527231867 특별시
9.983967527231867 쉐보레
9.983967527231867 쿠아리
9.983967527231867 ene
9.983967527231867 픽게임
9.983967527231867 술집이
9.983967527231867 술이다
9.983967527231867 씻겨
9.983967527231867 ael
9.983967527231867 순간들
9.983967527231867 프론트
9.983967527231867 백종
9.9

10.138118207059126 테스터
10.138118207059126 )',
10.138118207059126 /25
10.138118207059126 78)
10.138118207059126 유급
10.138118207059126 suit
10.138118207059126 걸그룹
10.138118207059126 솟
10.138118207059126 ould
10.138118207059126 투르크
10.138118207059126 시체
10.138118207059126 책추
10.138118207059126 타운홀
10.138118207059126 재적
10.138118207059126 서윤후
10.138118207059126 캠퍼
10.138118207059126 vel
10.138118207059126 한다든지
10.138118207059126 리포
10.138118207059126 EV
10.138118207059126 aly
10.138118207059126 구구절
10.138118207059126 대대
10.138118207059126 너스
10.138118207059126 블레드
10.138118207059126 밀접
10.138118207059126 oci
10.138118207059126 마네오
10.138118207059126 두줄
10.138118207059126 수기
10.138118207059126 리틱
10.138118207059126 스탯
10.138118207059126 Press
10.138118207059126 포지
10.138118207059126 brunch
10.138118207059126 ridge
10.138118207059126 pecial
10.138118207059126 교정교
10.138118207059126 너럴
10.138118207059126 불입
10.138118207059126 끊
10.138118207059126 뱃살
10.138118207059126 Sal
10.138118207059126 해

10.54358331516729 떤
10.54358331516729 TC
10.54358331516729 휩
10.54358331516729 after
10.54358331516729 되짚
10.54358331516729 암묵
10.54358331516729 하시다면
10.54358331516729 늙
10.54358331516729 설상가
10.54358331516729 보조배
10.54358331516729 르담
10.54358331516729 낄
10.54358331516729 갤
10.54358331516729 플랜트
10.54358331516729 뺏
10.54358331516729 언젠
10.54358331516729 두도
10.54358331516729 오징
10.54358331516729 gine
10.54358331516729 성게
10.54358331516729 저래
10.54358331516729 흙
10.54358331516729 임기
10.54358331516729 유후
10.54358331516729 얼버
10.54358331516729 왠
10.54358331516729 론하고
10.54358331516729 멈
10.54358331516729 조다
10.54358331516729 헛헛
10.54358331516729 촬
10.54358331516729 캠페
10.54358331516729 oug
10.54358331516729 통령
10.54358331516729 날로그
10.54358331516729 찡
10.54358331516729 일비재
10.54358331516729 굵
10.54358331516729 £
10.54358331516729 오프
10.54358331516729 업인
10.54358331516729 똘
10.54358331516729 lean
10.54358331516729 곰곰
10.54358331516729 위적으로
10.54358331516729 트럴
10.54358331516729 테랑
10.543583

11.236730495727235 히려
11.236730495727235 투리
11.236730495727235 탱크
11.236730495727235 지둥
11.236730495727235 임팩
11.236730495727235 엠씨
11.236730495727235 튕
11.236730495727235 tsy
11.236730495727235 터면
11.236730495727235 inanc
11.236730495727235 아슬
11.236730495727235 이지유
11.236730495727235 예기
11.236730495727235 팎
11.236730495727235 고루
11.236730495727235 저씨
11.236730495727235 use
11.236730495727235 뤠이첼
11.236730495727235 SALES
11.236730495727235 (7),
11.236730495727235 GOAL
11.236730495727235 캄
11.236730495727235 뗄
11.236730495727235 물며
11.236730495727235 부랴
11.236730495727235 염없이
11.236730495727235 닐라
11.236730495727235 숱
11.236730495727235 ety
11.236730495727235 슷
11.236730495727235 뻣
11.236730495727235 ationship
11.236730495727235 ustom
11.236730495727235 뺐
11.236730495727235 현듯
11.236730495727235 뿍
11.236730495727235 폈다
11.236730495727235 정성정성
11.236730495727235 odel
11.236730495727235 무수
11.236730495727235 윽고
11.236730495727235 숱
11.236730495727235 (0.
11.236730495727235 자지
11.23673049

4.253404029898023 회사
4.3200154753736255 조직
4.4786359912995035 연봉
4.624017643603171 업무
4.819181553308352 휴가
4.951663933656482 회식
4.954463748831229 출장
4.9695299471858725 커리어
5.143160725682099 성과
5.270583756603543 여행
5.301836300107647 기업
5.625428873865199 교육
5.672210088404541 사회
5.695466950568808 목표

In [81]:
idf_

,IDF
0,11.929878
1,11.929878
2,11.929878
3,4.278758
4,7.365529
...,...
49995,10.831265
49996,10.320440
49997,10.831265
49998,9.850436


In [155]:
idf_['included'] = (idf_['IDF'] > 7)

In [156]:
filtered_ids = []
for i in tqdm.tqdm(range(len(sentences_to_ids))):
    filtered_ids.append([])
    
    for word_id in sentences_to_ids[i]:
        if idf_.loc[word_id, 'included'] == True:
            filtered_ids[i].append(word_id)
            

100%|██████████| 151733/151733 [00:45<00:00, 3320.88it/s]


* (4.2, 8)

In [118]:
test_i = 1
print(sp.decode(filtered_ids[test_i]))
print(sentences[test_i])

첫 직장에서 했던 첫 회식은 신입 직원 신입 직원지만 주인공이라 찌그러져
첫 직장에서 했던 첫 회식은 신입 직원 환영회.말은 신입 직원 환영회지만 주인공이라 할 나는 구석에 찌그러져 있었다.


* (6, )

In [145]:
test_i = [1, 40, 950, 5966, 10000, 112121]
for i in test_i:
    print('original: ', sp.decode(filtered_ids[i]))
    print('generated: ', sentences[i])
    print('-----------------------------------------------------------------')

original:  직장에서 회식은 신입 환영회말은 신입 환영회 주인공 구석에 찌그러져
generated:  첫 직장에서 했던 첫 회식은 신입 직원 환영회.말은 신입 직원 환영회지만 주인공이라 할 나는 구석에 찌그러져 있었다.
-----------------------------------------------------------------
original:  단어는 그분의 문장은 그분의 이름을 포함해서 추켜 세워주면서 시작한다
generated:  특히 첫 단어는 그분의 성이기 때문에 첫 문장은 그분의 이름을 포함해서 최대한 추켜 세워주면서 시작한다.
-----------------------------------------------------------------
original:  어이없는 유형의 다음날 자기가 저질렀는지 기억하지 못한다는 거예요
generated:  가장 어이없는 건, 이 유형의 사람들은 다음날 자기가 어떤 일을 저질렀는지 잘 기억하지 못한다는 거예요.
-----------------------------------------------------------------
original:  이상하게도 대표님이 잡아먹으려고 아닌데 대표님이 불편하다
generated:  참 이상하게도 대표님이 나를 잡아먹으려고 하는 것도 아닌데 대표님이 있으면 좀 불편하다.
-----------------------------------------------------------------
original:  함께 팀의 분위기가 좋으면 성과도 향상 생각은 누구나 상식 선에서
generated:  ‘함께 일하는 팀의 분위기가 좋으면 성과도 향상될 것’이란 생각은 누구나 상식 선에서 할 수 있다.
-----------------------------------------------------------------
original:  모터스포츠 활동을 쌓은 기술력을 바탕으로 나은 미래와 자동차를 만들기 위해서다
generated:  모터스포츠 활동을 통해 쌓은

* (6, 9)

In [150]:
test_i = [1, 40, 950, 5966, 10000, 112121]
for i in test_i:
    print('original: ', sp.decode(filtered_ids[i]))
    print('generated: ', sentences[i])
    print('-----------------------------------------------------------------')

original:  직장에서 회식은 신입말은 신입 주인공 구석에 찌그러져
generated:  첫 직장에서 했던 첫 회식은 신입 직원 환영회.말은 신입 직원 환영회지만 주인공이라 할 나는 구석에 찌그러져 있었다.
-----------------------------------------------------------------
original:  단어는 그분의 문장은 그분의 이름을 포함해서 추켜 세워주면서 시작한다
generated:  특히 첫 단어는 그분의 성이기 때문에 첫 문장은 그분의 이름을 포함해서 최대한 추켜 세워주면서 시작한다.
-----------------------------------------------------------------
original:  유형의 다음날 자기가는지 못한다는 거예요
generated:  가장 어이없는 건, 이 유형의 사람들은 다음날 자기가 어떤 일을 저질렀는지 잘 기억하지 못한다는 거예요.
-----------------------------------------------------------------
original:  이상하게도 대표님이 잡아먹으려고 아닌데 대표님이 불편하다
generated:  참 이상하게도 대표님이 나를 잡아먹으려고 하는 것도 아닌데 대표님이 있으면 좀 불편하다.
-----------------------------------------------------------------
original:  함께 팀의 분위기가 좋으면 성과도 향상 생각은 누구나 상식 선에서
generated:  ‘함께 일하는 팀의 분위기가 좋으면 성과도 향상될 것’이란 생각은 누구나 상식 선에서 할 수 있다.
-----------------------------------------------------------------
original:  모터스포츠 활동을 쌓은 바탕으로 나은 자동차를 만들기 위해서다
generated:  모터스포츠 활동을 통해 쌓은 기술력을 바탕으로 보다 나은 미래와 더 좋은 자동차를 

* (7, 8)

In [154]:
test_i = [1, 40, 950, 5966, 10000, 112121]
for i in test_i:
    print('original: ', sp.decode(filtered_ids[i]))
    print('generated: ', sentences[i])
    print('-----------------------------------------------------------------')

original:  주인공 찌그러
generated:  첫 직장에서 했던 첫 회식은 신입 직원 환영회.말은 신입 직원 환영회지만 주인공이라 할 나는 구석에 찌그러져 있었다.
-----------------------------------------------------------------
original:  단어는켜 세워
generated:  특히 첫 단어는 그분의 성이기 때문에 첫 문장은 그분의 이름을 포함해서 최대한 추켜 세워주면서 시작한다.
-----------------------------------------------------------------
original:  
generated:  가장 어이없는 건, 이 유형의 사람들은 다음날 자기가 어떤 일을 저질렀는지 잘 기억하지 못한다는 거예요.
-----------------------------------------------------------------
original:  먹으려고
generated:  참 이상하게도 대표님이 나를 잡아먹으려고 하는 것도 아닌데 대표님이 있으면 좀 불편하다.
-----------------------------------------------------------------
original:  함께 성과도 향상
generated:  ‘함께 일하는 팀의 분위기가 좋으면 성과도 향상될 것’이란 생각은 누구나 상식 선에서 할 수 있다.
-----------------------------------------------------------------
original:  쌓은
generated:  모터스포츠 활동을 통해 쌓은 기술력을 바탕으로 보다 나은 미래와 더 좋은 자동차를 만들기 위해서다.
-----------------------------------------------------------------


* (7, )

In [157]:
test_i = [1, 40, 950, 5966, 10000, 112121]
for i in test_i:
    print('original: ', sp.decode(filtered_ids[i]))
    print('generated: ', sentences[i])
    print('-----------------------------------------------------------------')

original:  환영회말은 환영회 주인공 구석에 찌그러
generated:  첫 직장에서 했던 첫 회식은 신입 직원 환영회.말은 신입 직원 환영회지만 주인공이라 할 나는 구석에 찌그러져 있었다.
-----------------------------------------------------------------
original:  단어는 그분의 문장은 그분의 포함해서켜 세워주면서
generated:  특히 첫 단어는 그분의 성이기 때문에 첫 문장은 그분의 이름을 포함해서 최대한 추켜 세워주면서 시작한다.
-----------------------------------------------------------------
original:  어이없는 유형의 저질렀 기억하지 못한다는
generated:  가장 어이없는 건, 이 유형의 사람들은 다음날 자기가 어떤 일을 저질렀는지 잘 기억하지 못한다는 거예요.
-----------------------------------------------------------------
original:  이상하게도 대표님이먹으려고 대표님이 불편하다
generated:  참 이상하게도 대표님이 나를 잡아먹으려고 하는 것도 아닌데 대표님이 있으면 좀 불편하다.
-----------------------------------------------------------------
original:  함께 좋으면 성과도 향상 상식 선에서
generated:  ‘함께 일하는 팀의 분위기가 좋으면 성과도 향상될 것’이란 생각은 누구나 상식 선에서 할 수 있다.
-----------------------------------------------------------------
original:  모터스포츠 쌓은 기술력을 미래와 자동차를 위해서다
generated:  모터스포츠 활동을 통해 쌓은 기술력을 바탕으로 보다 나은 미래와 더 좋은 자동차를 만들기 위해서다.
---------------------------------------